In [12]:
import re
from graphviz import Digraph
import typing
import IPython
from __future__ import annotations

Преобразованная грамматика


```
<выражение> ->
    <арифметическое выражение> <знак операции отношения> <арифметическое выражение>

<арифметическое выражение> ->
    [ <знак операции типа сложение> ] <терм> { <знак операции типа сложение> <терм> }

<терм> ->
    <множитель> [ <знак операции типа умножение> <терм> ]

<множитель> ->
    <первичное выражение> [ ^ <множитель> ]

<первичное выражение> ->
    <число> |
    <идентификатор> |
    ( <арифметическое выражение> )

<знак операции типа сложение> ->
    + | -

<знак операции типа умножение> ->
    * | / | %

<знак операции отношения> ->
    < | <= | = | >= | > | <>
```

Преобразованная грамматика 2


```
<выражение> ->
    <арифметическое выражение> <знак операции отношения> <арифметическое выражение>

<арифметическое выражение 2> ->
    <терм> <знак операции типа сложение> <арифметическое выражение 2> 
    <терм>

<арифметическое выражение> ->
    <знак операции типа сложение> <арифметическое выражение 2> |
    <арифметическое выражение 2>

<терм> ->
    <множитель> <знак операции типа умножение> <терм> |
    <множитель>

<множитель> ->
    <первичное выражение> [ ^ <множитель> ]

<первичное выражение> ->
    <число> |
    <идентификатор> |
    ( <арифметическое выражение> )

<знак операции типа сложение> ->
    + | -

<знак операции типа умножение> ->
    * | / | %

<знак операции отношения> ->
    < | <= | = | >= | > | <>
```

In [13]:
from __future__ import annotations


class TreeNode:
    data = ""
    children: list[TreeNode]
    from_ = -1
    to_ = -1

    attrubs = ""

    def __init__(self, data) -> None:
        self.data = data
        self.children = []
        self.attrubs = []

    def add_child(self, node: TreeNode):
        if not len(self.children):
            self.from_ = node.from_
        self.to_ = node.to_

        if node.data in ["<proxy>"]:
            for ch in node.children:
                self.add_child(ch)
            return

        self.children.append(node)
    
    def recurse(self, tree=None, parent=None, id="main"):
        if not tree:
            tree = Digraph()
        tree.node(id, str(self.data))
        if parent:
            tree.edge(parent, id)

        for i, child in enumerate(self.children):
            child.recurse(tree, id, id + "." + str(i))

        return tree

In [14]:
def run_tests(func, tests: list[tuple[str, str]], full=False, cmp=lambda a, b: repr(a) == repr(b)):
    output = "<table><thead><tr><td><div style='width:100px'>Input</div></td><td><div style='width:200px'>Expected Output</div></td><td><div style='width:200px'>Actual Output</div></td><td>Comment</td></tr></thead><tbody>"
    for test in tests:
        result = func(test[0])
        is_correct = cmp(result, test[1])
        if not is_correct or full:
            output += "<tr><td>\n\n{:}\n</td><td>\n\n{:}\n</td><td>\n\n{:}</td><td>\n\n{:}\n</td></tr>\n".format(
                repr(test[0]),
                repr(test[1]),
                repr(result),
                "OK" if is_correct else "ERROR",
            )
    output += "</tbody></table>"
    return output

In [15]:
def make_parse_lexem(lexem: str):
    def parse_lexem(lexems: list[str], startPos: int):
        if startPos >= len(lexems):
            return None
        if lexems[startPos] == lexem:
            tree = TreeNode(lexem)
            tree.from_ = startPos
            tree.to_ = startPos + 1
            return tree
        return None

    return parse_lexem


def make_choice(rules):
    def combinator(lexems: typing.List[str], startPos: int):
        for rule in rules:
            node = rule(lexems, startPos)
            if not node:
                continue
            return node

        return None

    return combinator


def make_combine(rules, actions = None):
    def combinator(lexems: list[str], startPos: int):
        tree = TreeNode("<proxy>")
        curPos = startPos
        for rule in rules:
            node = rule(lexems, curPos)
            if not node:
                return None

            tree.add_child(node)
            curPos = node.to_
        return tree

    def action_wrapper(lexems: list[str], startPos: int):
        tree = combinator(lexems, startPos)
        if tree and actions:
            tree.attrubs = actions(*tree.children)
        return tree

    return action_wrapper


def parse_rel_op(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_choice(
        [
            make_parse_lexem("<"),
            make_parse_lexem("<="),
            make_parse_lexem("="),
            make_parse_lexem(">="),
            make_parse_lexem(">"),
            make_parse_lexem("!="),
        ]
    )(lexems, startPos)
    if not tree:
        return None

    # tree.data = "<операция отношения>"
    return tree


def parse_mult_op(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_choice(
        [
            make_parse_lexem("*"),
            make_parse_lexem("/"),
            make_parse_lexem("%"),
        ]
    )(lexems, startPos)
    if not tree:
        return None

    # tree.data = "<операция типа умножение>"
    return tree


def parse_sum_op(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_choice(
        [
            make_parse_lexem("+"),
            make_parse_lexem("-"),
        ]
    )(lexems, startPos)
    if not tree:
        return None

    # tree.data = "<операция типа сложение>"
    return tree


def parse_id(lexems: list[str], startPos: int) -> TreeNode | None:
    if startPos >= len(lexems):
        return None
    tree = TreeNode("<идентификатор>")

    data = lexems[startPos]
    if not data.isalpha():
        return None

    node = TreeNode(data)
    node.from_ = startPos
    node.to_ = startPos + 1

    tree.add_child(node)
    tree.attrubs = [data]

    return tree


def parse_num(lexems: list[str], startPos: int) -> TreeNode | None:
    if startPos >= len(lexems):
        return None

    tree = TreeNode("<число>")

    try:
        data = float(lexems[startPos])
    except:
        return None

    node = TreeNode(data)
    node.from_ = startPos
    node.to_ = startPos + 1

    tree.add_child(node)
    tree.attrubs = [data]

    return tree


def parse_first_exp(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_choice(
        [
            make_combine(
                [make_parse_lexem("("), parse_arythmetical_stmt, make_parse_lexem(")")],
                actions=lambda a, b, c: b.attrubs,
            ),
            parse_id,
            parse_num,
        ]
    )(lexems, startPos)
    if not tree:
        return None

    tree.data = "<первичное выражение>"
    return tree


def parse_multiplier(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_choice(
        [
            make_combine(
                [parse_first_exp, make_parse_lexem("^"), parse_multiplier],
                actions=lambda a, b, c: a.attrubs + c.attrubs + [b.data],
            ),
            parse_first_exp,
        ]
    )(lexems, startPos)
    if not tree:
        return None

    tree.data = "<множитель>"
    return tree


def parse_term(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_choice(
        [
            make_combine(
                [parse_multiplier, parse_mult_op, parse_term],
                actions=lambda a, b, c: a.attrubs + c.attrubs + [b.data],
            ),
            parse_multiplier,
        ]
    )(lexems, startPos)
    if not tree:
        return None

    tree.data = "<терм>"
    return tree


def parse_arythmetical_stmt_2(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_choice(
        [
            make_combine(
                [parse_term, parse_sum_op, parse_arythmetical_stmt_2],
                actions=lambda a, b, c: a.attrubs + c.attrubs + [b.data],
            ),
            parse_term,
        ],
    )(lexems, startPos)
    if not tree:
        return None

    tree.data = "<арифметическое выражение 2>"
    return tree


def parse_arythmetical_stmt(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_choice(
        [
            make_combine(
                [parse_sum_op, parse_arythmetical_stmt_2],
                actions=lambda a, b: b.attrubs + [a.data],
            ),
            parse_arythmetical_stmt_2,
        ],
    )(lexems, startPos)
    if not tree:
        return None

    tree.data = "<арифметическое выражение>"
    return tree


def parse_expr(lexems: list[str], startPos: int) -> TreeNode | None:
    tree = make_combine(
        [
            parse_arythmetical_stmt,
            parse_rel_op,
            parse_arythmetical_stmt,
        ],
        actions=lambda a, b, c: a.attrubs + c.attrubs + [b.data],
    )(lexems, startPos)
    if not tree:
        return None
    tree.data = "<выражение>"

    return tree

In [16]:
report = run_tests(
    lambda x: parse_expr(x, 0),
    [(["1234"],None,),
        (["1", "<>", "2"],None,),
        (["1", "+", "3", "=", "4"],{},),
        (["105", "+", "3", "%", "(", "2", "*", "4", "^", "2", ")", "=", "4"],{},),
        (["x", "^", "2", "-", "x", "+", "1", ">", "e", "^", "(", "x", "^", "2", ")"],{},),
    ],
    True,
    lambda a, b: a is not None or repr(a) == repr(b),
)


IPython.display.Markdown(report)

<table><thead><tr><td><div style='width:100px'>Input</div></td><td><div style='width:200px'>Expected Output</div></td><td><div style='width:200px'>Actual Output</div></td><td>Comment</td></tr></thead><tbody><tr><td>

['1234']
</td><td>

None
</td><td>

None</td><td>

OK
</td></tr>
<tr><td>

['1', '<>', '2']
</td><td>

None
</td><td>

None</td><td>

OK
</td></tr>
<tr><td>

['1', '+', '3', '=', '4']
</td><td>

{}
</td><td>

<__main__.TreeNode object at 0x7fb10db8d3a0></td><td>

OK
</td></tr>
<tr><td>

['105', '+', '3', '%', '(', '2', '*', '4', '^', '2', ')', '=', '4']
</td><td>

{}
</td><td>

<__main__.TreeNode object at 0x7fb10db8d2b0></td><td>

OK
</td></tr>
<tr><td>

['x', '^', '2', '-', 'x', '+', '1', '>', 'e', '^', '(', 'x', '^', '2', ')']
</td><td>

{}
</td><td>

<__main__.TreeNode object at 0x7fb10db8dd00></td><td>

OK
</td></tr>
</tbody></table>

In [17]:
# Определение терминалов
numeric = r'\d+'
alphabet = r'a-zA-Z_\w'
blanc_space = r'( |\t|\n|\r)'
symbols = r'(\+|\-|\*|/|%|<|=|>|(|)|{|}|;|^)'
double_symbols = r'(<=|>=|!=)'

def lexical_analyzer(input_str: str):
    tokens = []
    cur = 0
    while cur < len(input_str):
        if re.findall(f'{double_symbols}', input_str[cur : min(cur + 2, len(input_str))]):
            tokens += [input_str[cur : min(cur + 2, len(input_str))]]
            cur += 2
        elif re.findall(f'{blanc_space}', input_str[cur]):
            cur += 1
        elif input_str[cur] in ['+', '-', '*', '/', '%', '<', '=', '>', '(', ')', '{', '}', ';', '^']:
            tokens += [input_str[cur]]
            cur += 1
        else:
            start = cur
            while cur < len(input_str) and (
                re.findall(f'{numeric}', input_str[cur])
                or input_str[cur].isalpha()
                or input_str[cur] in ["_", "."]
        ):	
                cur += 1
            tokens += [input_str[start:cur]]
            if start == cur:
                print("Ошибка анализатора")
                break
    
    return tokens


input_string = '22 + 6^2 - 14 > 5'
result = lexical_analyzer(input_string)
print(result)

['22', '+', '6', '^', '2', '-', '14', '>', '5']


In [18]:
code = "5 + 5 + 7 * 9 - (14 + 12) ^ 2 < 20"
print(lexical_analyzer(code))
tree = parse_expr(lexical_analyzer(code), 0)
tree.recurse()
tree.recurse().render('parse_tree', format='png', cleanup=True)

['5', '+', '5', '+', '7', '*', '9', '-', '(', '14', '+', '12', ')', '^', '2', '<', '20']


'parse_tree.png'

### Обратная польская нотация

In [19]:
def test(input, output):
      tree = parse_expr(lexical_analyzer(input), 0)
      res = " ".join(map(str,tree.attrubs))
      return "|" + input + "|" + output + "|" + res + "|" + str(output == res) + "|"

tests = [
        (
            "10 > 2",
            "10.0 2.0 >",
        ),
        (
            "5 + 66 != 2",
            "5.0 66.0 + 2.0 !="
        ),
        (
            "2 * 89 ^ 1 != 2",
            "2.0 89.0 1.0 ^ * 2.0 !="
        ),
        (
            "x ^ 2 - x + 1 < e^(x ^ 2)",
            "x 2.0 ^ x 1.0 + - e x 2.0 ^ ^ <"
        ),
        (
              "(а + b)^2 - 15 % 8 = 0",
              "а b + 2.0 ^ 15.0 8.0 % - 0.0 ="
        )
    ]

IPython.display.Markdown("|Input|Expected result|Actual result|Сonclusion |\n|---|---|---|---|\n" + "\n".join(list(map(lambda x: test(x[0], x[1]), tests))))

|Input|Expected result|Actual result|Сonclusion |
|---|---|---|---|
|10 > 2|10.0 2.0 >|10.0 2.0 >|True|
|5 + 66 != 2|5.0 66.0 + 2.0 !=|5.0 66.0 + 2.0 !=|True|
|2 * 89 ^ 1 != 2|2.0 89.0 1.0 ^ * 2.0 !=|2.0 89.0 1.0 ^ * 2.0 !=|True|
|x ^ 2 - x + 1 < e^(x ^ 2)|x 2.0 ^ x 1.0 + - e x 2.0 ^ ^ <|x 2.0 ^ x 1.0 + - e x 2.0 ^ ^ <|True|
|(а + b)^2 - 15 % 8 = 0|а b + 2.0 ^ 15.0 8.0 % - 0.0 =|а b + 2.0 ^ 15.0 8.0 % - 0.0 =|True|